# Mixing Ingredients—Python Dictionaries
<img src="misc/4_ingredients on table and mixing bowl.png" width="100%" />

In Lesson 2 we read the ingredients/steps.  In Lesson 3, we parsed those ingredients to gain some understanding of what they do. Now we will take the steps and mix the ingredients. In doing so we will gain a deeper understanding of advanced data structures, like dictionaries.

In this chapter we read over the steps and operate on the ingredients.

> read in the steps

In [ ]:
import pickle
recipe_as_dict = pickle.load(open("data.pickle", "rb"))

## 4.1 Logging and control

> Our program is getting more complex; we will start using the built-in Python logging to help us get feedback as we run.

In [ ]:
import logging as log
from pprint import pformat
logger = log.getLogger()
logger.setLevel(log.DEBUG)

In [ ]:
ch = log.StreamHandler()
ch.setFormatter(log.Formatter('%(levelname)s: %(message)s'))
logger.removeHandler(0)
logger.addHandler(ch)
log.info("who loves Pie?")
log.debug("i am a debug message")

Let's break out the filling and crust steps:

## 4.2 Start with the filling

Inspect the filling_steps:

A couple of observations:

 * There are three parts
 * There is a bunch of `\n`, which means line break, but they aren't important
 * What may be important is the sentences in each

Let's break this up into a list of list based on sentences.


We want to identify the keywords that indicate mixing:

 * "mix"
 * "pour"
 * "sprinkle"
 * "dot"

From Lesson 3, we have a list of ingredients already broken up into quantities and units.

The ingredients will live in three states:

 * The preparation counter (the_counter)
 * The mixing bowl (mixing_bowl)
 * The pie plate (pie_plate)
 
We need to move ingredients from state to state from step to step and keep track of the ingredients. Later we will need to interrupt the pie-making process by refilling the ingredients on the preparation counter.

> We start out with those steps and identify which ones require action.

In [ ]:
the_counter = {}
mixing_bowl = {}
pie_plate = {}



We need to break it out into functions:

 * A function to test whether it has a mix operation (has_mix_operation)
 * A function to do the mixing (mix)
 
Before we pick it up with the actual steps, we prototype each function.


The `mix()` function is what does the mixing (duh), but it also uses dictionaries (dicts) as input. But first a quick tutorial on dicts.

## 4.3 Notes on dicts

A couple of notes on dictionaries:

 * Dicts are not sorted, so they don't retain order
 * Keys must be unique
 * Dicts point to data values so they retain changes
 * The types used for keys and values do not need to be same even inside same dict

Some examples of this...

```python
>>> the_dict = {'a':2, 'b':2, 'c':2, 'd':4, 'e':4}
>>> print(the_dict)
{'d': 4, 'b': 2, 'c': 2, 'a': 2, 'e': 4}
>>> def changeone(passed_dict):
...    passed_dict['d'] = 1337
>>> changeone(the_dict)
>>> print(the_dict)
{'d': 1337, 'b': 2, 'c': 2, 'a': 2, 'e': 4}
```

Now back to our `mix()` function:

In [ ]:
from output import LargeItem, IngredientBase, DrySolid, Liquid
from fractions import Fraction

In [ ]:
import pickle
shopping_list = pickle.load(open("shopping_list.pickle", "rb"))

## 4.4 Reusable code

## 4.5 Putting it all together.

 * Fill the counter with items and move to the mixing_bowl (pie_plate will remain empty for now)
 * Iterate over each steps
 * Operate properly on each




## 4.6 Code-reuse

We can cut and past the code above and apply to crust. We will find out in the following chapter how this code can better be organized as functions.